In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor, RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, ComplementNB  # ComplementNB ging nicht, da dieser mit negativen Werten nicht klarkam

# Constant variables
RANDOM_STATE = 42
CV = 5
N_JOBS = -1

# Daten einlesen und vorverarbeiten

In [11]:
with open('data/data_scaled.pickle', 'rb') as f:
    df_scaled = pickle.load(f)
df_scaled.describe()

C:\Users\sfremerey\AppData\Local\Temp\ipykernel_12088\584721221.py:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df_scaled = pickle.load(f)


,DATE,GOLD_CLOSE,GOLD_VOLUME,SP_CLOSE,SP_VOLUME,DJ_CLOSE,DJ_VOLUME,EG_CLOSE,EG_VOLUME,EU_CLOSE,...,GDX_100,USO_15,USO_100,MONTH,QUARTER,FEAR_GREED,EFFR,GPRD,CPI,LABEL_GOLD
count,1718,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,...,1618.000000,1703.000000,1618.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1707.000000,1718.000000
mean,2015-07-06 19:29:15.995343360,0.366837,0.075239,0.485898,0.170939,0.424765,0.190005,0.329991,0.098171,0.478699,...,-0.080589,-0.040328,-0.087351,0.545693,0.627910,0.505850,0.209196,0.272716,0.329038,-0.035506
min,2011-12-15 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.912176,-1.285804,-1.037401,0.083333,0.250000,0.000000,0.000000,0.000000,0.000000,-1.000000
25%,2013-10-03 06:00:00,0.212727,0.042375,0.289154,0.095986,0.247601,0.094059,0.147403,0.054172,0.231463,...,-0.189903,-0.186442,-0.197444,0.333333,0.500000,0.340000,0.029661,0.196216,0.109626,-1.000000
50%,2015-07-18 12:00:00,0.291273,0.064812,0.491892,0.149907,0.388171,0.125866,0.256169,0.081270,0.409783,...,-0.052595,-0.012858,-0.027271,0.583333,0.750000,0.530000,0.050847,0.254719,0.192513,0.000000
75%,2017-04-09 06:00:00,0.382574,0.094324,0.672861,0.223743,0.603162,0.286089,0.446591,0.119259,0.752326,...,0.040048,0.121348,0.084541,0.833333,1.000000,0.680000,0.368644,0.332301,0.508021,1.000000
max,2018-12-31 00:00:00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.546824,0.867918,0.332710,1.000000,1.000000,0.950000,1.000000,1.000000,1.000000,1.000000
std,NaN,0.239861,0.053311,0.252623,0.102737,0.257898,0.135944,0.263567,0.073775,0.283436,...,0.250000,0.250000,0.250000,0.285445,0.278662,0.224843,0.266818,0.114884,0.294851,0.799983


In [12]:
with open('data/data_raw.pickle', 'rb') as f:
    df_raw = pickle.load(f)
df_raw.describe()

C:\Users\sfremerey\AppData\Local\Temp\ipykernel_12088\2863861877.py:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df_raw = pickle.load(f)


,DATE,GOLD_CLOSE,GOLD_VOLUME,SP_CLOSE,SP_VOLUME,DJ_CLOSE,DJ_VOLUME,EG_CLOSE,EG_VOLUME,EU_CLOSE,...,GDX_100,USO_15,USO_100,MONTH,QUARTER,FEAR_GREED,EFFR,GPRD,CPI,LABEL_GOLD
count,1718,1718.000000,1.718000e+03,1718.000000,1.718000e+03,1718.000000,1.718000e+03,1718.000000,1.718000e+03,1718.000000,...,1618.000000,1703.000000,1618.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1707.000000,1718.000000
mean,2015-07-06 19:29:15.995343360,127.319482,8.446327e+06,204.491222,1.098026e+08,18164.119043,1.779131e+08,28.209302,1.136074e+06,1.208494,...,-2.007281,-0.250910,-1.601514,6.548312,2.511641,50.584983,0.533702,101.689703,150.412068,-0.035506
min,2011-12-15 00:00:00,100.500000,1.501600e+06,120.290001,2.785650e+07,11766.259770,8.410000e+06,2.800000,1.645000e+05,1.038700,...,-22.719999,-8.000000,-19.020003,1.000000,1.000000,0.000000,0.040000,7.060000,125.800000,-1.000000
25%,2013-10-03 06:00:00,116.052502,5.412925e+06,170.397500,7.387085e+07,15495.665280,9.232000e+07,14.150000,7.006250e+05,1.120800,...,-4.729999,-1.160000,-3.620002,4.000000,2.000000,34.000000,0.110000,75.145000,134.000000,-1.000000
50%,2015-07-18 12:00:00,121.795002,7.483900e+06,205.529999,9.972020e+07,17612.939455,1.206950e+08,22.524999,9.688000e+05,1.184050,...,-1.309998,-0.080002,-0.500000,7.000000,3.000000,53.000000,0.160000,95.445000,140.200000,0.000000
75%,2017-04-09 06:00:00,128.470001,1.020795e+07,236.889996,1.351161e+08,20851.157715,2.636300e+08,37.187501,1.344775e+06,1.305550,...,0.997500,0.754999,1.550000,10.000000,4.000000,68.000000,0.910000,122.365000,163.800000,1.000000
max,2018-12-31 00:00:00,173.610001,9.380420e+07,293.579987,5.072443e+08,26828.390630,9.005100e+08,79.800003,1.006120e+07,1.393400,...,13.620000,5.400002,6.099999,12.000000,4.000000,95.000000,2.400000,354.050000,200.600000,1.000000
std,NaN,17.536269,4.920731e+06,43.776999,4.925110e+07,3884.495887,1.212753e+08,20.294635,7.301289e+05,0.100535,...,6.226869,1.555447,4.583572,3.425341,1.114647,22.484254,0.629690,39.863463,22.054850,0.799983


In [13]:
# Drop columns in scaled data
_symbols = ["GOLD", "SP", "DJ", "EG", "EU", "OF", "OS", "SF", "USB", "PLT", "PLD", "USDI", "GDX", "USO"]
for k in _symbols:
    df_scaled = df_scaled.drop(columns=[k+"_CLOSE"])

df_scaled.dropna(axis=0, inplace=True)
df_scaled = df_scaled.drop(columns=["DATE"])

df_scaled.head(10)

,GOLD_VOLUME,SP_VOLUME,DJ_VOLUME,EG_VOLUME,OF_VOLUME,SF_VOLUME,USDI_VOLUME,GDX_VOLUME,USO_VOLUME,DIFF_GOLD,...,GDX_100,USO_15,USO_100,MONTH,QUARTER,FEAR_GREED,EFFR,GPRD,CPI,LABEL_GOLD
100,0.073497,0.261116,0.156888,0.085625,0.319880,0.354067,0.151513,0.034754,0.031786,3.880005,...,-0.425174,-0.522358,-0.034907,0.416667,0.5,0.25,0.046610,0.157094,0.029412,1
101,0.128779,0.283807,0.151082,0.079784,0.337193,0.291080,0.211964,0.075017,0.038676,5.559997,...,-0.462913,-0.501464,-0.031089,0.416667,0.5,0.23,0.050847,0.169630,0.029412,1
102,0.128414,0.375005,0.165508,0.124152,0.242252,0.352153,0.343724,0.087053,0.047971,6.050003,...,-0.527954,-0.657367,-0.060542,0.416667,0.5,0.20,0.050847,0.246693,0.029412,1
103,0.165208,0.374246,0.151014,0.118161,0.019596,0.528156,0.265270,0.105490,0.087979,5.020004,...,-0.514705,-0.687906,-0.122721,0.416667,0.5,0.14,0.050847,0.172944,0.029412,1
104,0.179617,0.459203,0.155835,0.145665,0.506472,0.487555,0.335668,0.114375,0.043375,3.899994,...,-0.474155,-0.707192,-0.143447,0.416667,0.5,0.12,0.050847,0.134961,0.029412,1
105,0.124510,0.608608,0.260408,0.107066,0.350532,0.443861,0.220580,0.075544,0.056544,0.899994,...,-0.439226,-0.822915,-0.209444,0.416667,0.5,0.12,0.050847,0.403729,0.029412,0
106,0.065446,0.312909,0.149871,0.033405,0.287232,0.324464,0.199356,0.038726,0.041244,2.350006,...,-0.384624,-0.687905,-0.181081,0.416667,0.5,0.16,0.050847,0.149889,0.029412,1
107,0.106184,0.353940,0.140388,0.060465,0.303268,0.383475,0.231367,0.070524,0.038900,3.529999,...,-0.394661,-0.827737,-0.218716,0.416667,0.5,0.15,0.050847,0.313208,0.029412,1
108,0.188000,0.369433,0.161170,0.148605,0.387405,0.427267,0.267722,0.117274,0.075665,6.250000,...,-0.294289,-0.951495,-0.261259,0.416667,0.5,0.15,0.046610,0.227903,0.029412,1
109,0.088789,0.290998,0.132721,0.113129,0.322846,0.367716,0.250771,0.074809,0.056181,5.129989,...,-0.196327,-0.830951,-0.225261,0.416667,0.5,0.14,0.046610,0.306579,0.029412,1


In [14]:
# Drop columns in raw data
_symbols = ["GOLD", "SP", "DJ", "EG", "EU", "OF", "OS", "SF", "USB", "PLT", "PLD", "USDI", "GDX", "USO"]
for k in _symbols:
    df_raw = df_raw.drop(columns=[k+"_CLOSE"])

df_raw.dropna(axis=0, inplace=True)
df_raw = df_raw.drop(columns=["DATE"])

df_raw.head(10)

,GOLD_VOLUME,SP_VOLUME,DJ_VOLUME,EG_VOLUME,OF_VOLUME,SF_VOLUME,USDI_VOLUME,GDX_VOLUME,USO_VOLUME,DIFF_GOLD,...,GDX_100,USO_15,USO_100,MONTH,QUARTER,FEAR_GREED,EFFR,GPRD,CPI,LABEL_GOLD
100,8285600,153032400,148370000,1011900,189450,72160,21690,12632900,4507100,3.880005,...,-10.590000,-3.250000,-0.640004,5,2,25,0.15,61.57,128.0,1
101,13388200,163910000,143190000,954100,199080,59330,30320,21789800,5259700,5.559997,...,-11.529999,-3.119999,-0.569999,5,2,23,0.16,65.92,128.0,1
102,13354500,207629300,156060000,1393200,146270,71770,49130,24527000,6275000,6.050003,...,-13.150002,-4.090000,-1.110000,5,2,20,0.16,92.66,128.0,1
103,16750700,207265500,143130000,1333900,22420,107620,37930,28720000,10645100,5.020004,...,-12.820000,-4.280003,-2.250000,5,2,14,0.16,67.07,128.0,1
104,18080700,247992900,147430000,1606100,293240,99350,47980,30740700,5773000,3.899994,...,-11.810002,-4.399998,-2.629997,5,2,12,0.16,53.89,128.0,1
105,12994200,319615900,240720000,1224100,206500,90450,31550,21909600,7211400,0.899994,...,-10.940002,-5.119999,-3.840000,5,2,12,0.16,147.15,128.0,0
106,7542400,177861100,142110000,495100,171290,66130,28520,13536300,5540200,2.350006,...,-9.580002,-4.279998,-3.319999,5,2,16,0.16,59.07,128.0,1
107,11302700,197531200,133650000,762900,180210,78150,33090,20767800,5284200,3.529999,...,-9.829998,-5.150001,-4.010002,5,2,15,0.16,115.74,128.0,1
108,18854500,204958400,152190000,1635200,227010,87070,38280,31400000,9300000,6.250000,...,-7.330001,-5.919998,-4.790001,5,2,15,0.15,86.14,128.0,1
109,9697100,167357600,126810000,1284100,191100,74940,35860,21742500,7171800,5.129989,...,-4.889999,-5.170002,-4.130001,5,2,14,0.15,113.44,128.0,1


In [15]:
y = df_scaled["LABEL_GOLD"]  # Für Klassifikation
# y = df_scaled["DIFF_GOLD"]  # Für Regression

df_scaled = df_scaled.drop(columns=["LABEL_GOLD"])
df_scaled = df_scaled.drop(columns=["DIFF_GOLD"])
# df_scaled = df_scaled.drop(columns=["FEAR_GREED"])
# df_scaled = df_scaled.drop(columns=["EFFR"])
# df_scaled = df_scaled.drop(columns=["GPRD"])
# df_scaled = df_scaled.drop(columns=["CPI"])


X_scaled = df_scaled

In [16]:
df_raw = df_raw.drop(columns=["LABEL_GOLD"])
df_raw = df_raw.drop(columns=["DIFF_GOLD"])

X_raw = df_raw

# MLPClassifier mit und ohne GridSearch auf skalierten Daten

In [17]:
X_scaled_train, X_scaled_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, shuffle=True)

X_scaled_train

,GOLD_VOLUME,SP_VOLUME,DJ_VOLUME,EG_VOLUME,OF_VOLUME,SF_VOLUME,USDI_VOLUME,GDX_VOLUME,USO_VOLUME,GOLD_15,...,GDX_15,GDX_100,USO_15,USO_100,MONTH,QUARTER,FEAR_GREED,EFFR,GPRD,CPI
1431,0.052317,0.066195,0.340590,0.091839,0.469617,0.075163,0.171056,0.132117,0.163105,-0.163061,...,-0.159418,-0.012848,0.128580,0.085632,0.916667,1.00,0.66,0.474576,0.132108,0.697861
697,0.054849,0.238785,0.148560,0.118979,0.455001,0.161274,0.387083,0.181445,0.089470,0.190953,...,0.269216,-0.243301,-0.183227,-0.545426,0.916667,1.00,0.59,0.025424,0.298222,0.155080
593,0.039449,0.089186,0.061910,0.044530,0.376097,0.082969,0.068997,0.110456,0.010466,0.371179,...,0.417020,0.109606,0.263590,0.244896,0.500000,0.50,0.93,0.025424,0.351422,0.132353
807,0.025513,0.178289,0.097444,0.051724,0.438875,0.001816,0.336929,0.112104,0.245633,-0.014483,...,0.099240,0.048178,0.326273,-0.202899,0.416667,0.50,0.58,0.038136,0.108649,0.188503
600,0.045907,0.051354,0.055409,0.039357,0.464260,0.002995,0.105702,0.189284,0.014834,0.332023,...,0.354731,0.131286,0.028931,0.200717,0.583333,0.75,0.87,0.025424,0.154414,0.139037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,0.062402,0.072721,0.286257,0.077581,0.423504,0.063872,0.265971,0.194590,0.135260,0.319686,...,0.372679,-0.316772,-0.038574,0.016363,0.083333,0.25,0.57,0.262712,0.345774,0.447861
1394,0.074184,0.060576,0.399249,0.095628,0.481483,0.064215,0.220370,0.127203,0.120252,0.215627,...,0.174198,0.025695,0.000000,-0.063269,0.750000,0.75,0.63,0.474576,0.139312,0.636364
960,0.057385,0.382398,0.141352,0.058514,0.280077,0.108793,0.104581,0.186521,0.397561,0.002146,...,0.072847,0.012848,-0.297342,-0.314711,1.000000,1.00,0.24,0.042373,0.106804,0.227273
1559,0.030309,0.066783,0.297881,0.078410,0.449950,0.053169,0.141496,0.090749,0.119334,-0.090649,...,0.006334,0.032119,0.094828,0.165264,0.416667,0.50,0.55,0.703390,0.310153,0.918449


In [18]:
parameters = {
    "solver":("lbfgs", "sgd", "adam"),
    "hidden_layer_sizes":((100,), (2,2), (4,4), (6,6), (8,8), (10,10), (2,2,2), (4,4,4), (6,6,6), (12,12,12), (24,24,24)),  #, (2,2), (4,4), (6,6), (8,8), (10,10),(2,2,2), (4,4,4), (6,6,6), (12,12,12), (24,24,24) 
    "activation":("identity", "logistic", "tanh", "relu"),
    "learning_rate":(("constant", "invscaling", "adaptive"))
}
model_mlpclassifier = MLPClassifier(max_iter=10000, random_state=RANDOM_STATE)
clf = GridSearchCV(model_mlpclassifier, parameters, n_jobs=N_JOBS, cv=CV)
clf.fit(X_scaled_train, y_train)
clf_results = pd.DataFrame(clf.cv_results_)
clf_results.sort_values(by="mean_test_score", ascending=False, inplace=True)
# 1st mean_test_score: 0.38
# 2nd mean_test_score: 0.39 (Parameter angepasst)
# 3rd mean_test_score: 0.754 (shuffle=True)
# 4th mean_test_score: 0.756 (mit skalierten MONTH und QUARTER)
clf_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_hidden_layer_sizes,param_learning_rate,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
200,48.479698,11.811635,0.014072,0.004684,tanh,"(100,)",constant,adam,"{'activation': 'tanh', 'hidden_layer_sizes': (...",0.750973,0.750973,0.757812,0.773438,0.750000,0.756639,0.008853,1
203,48.950372,12.188059,0.014516,0.003082,tanh,"(100,)",invscaling,adam,"{'activation': 'tanh', 'hidden_layer_sizes': (...",0.750973,0.750973,0.757812,0.773438,0.750000,0.756639,0.008853,1
206,48.506194,11.480668,0.013509,0.002433,tanh,"(100,)",adaptive,adam,"{'activation': 'tanh', 'hidden_layer_sizes': (...",0.750973,0.750973,0.757812,0.773438,0.750000,0.756639,0.008853,1
299,22.329469,2.097890,0.013126,0.005152,relu,"(100,)",constant,adam,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.712062,0.750973,0.718750,0.785156,0.726562,0.738701,0.026697,4
305,15.267072,1.870773,0.009785,0.004876,relu,"(100,)",adaptive,adam,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.712062,0.750973,0.718750,0.785156,0.726562,0.738701,0.026697,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,0.291392,0.022924,0.011726,0.001890,logistic,"(4, 4)",invscaling,sgd,"{'activation': 'logistic', 'hidden_layer_sizes...",0.291829,0.291829,0.289062,0.289062,0.289062,0.290169,0.001355,389
265,0.239332,0.021100,0.014542,0.003997,tanh,"(4, 4, 4)",invscaling,sgd,"{'activation': 'tanh', 'hidden_layer_sizes': (...",0.291829,0.291829,0.289062,0.285156,0.285156,0.288607,0.002993,393
301,0.472166,0.187288,0.014250,0.004525,relu,"(100,)",invscaling,sgd,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.287938,0.287938,0.273438,0.289062,0.292969,0.286269,0.006677,394
40,0.809868,0.106830,0.011417,0.001294,identity,"(8, 8)",invscaling,sgd,"{'activation': 'identity', 'hidden_layer_sizes...",0.260700,0.291829,0.292969,0.308594,0.269531,0.284725,0.017298,395


In [19]:
model_mlpclassifier_scaleddata = MLPClassifier(max_iter=10000, random_state=RANDOM_STATE, hidden_layer_sizes=(100,), activation="tanh", learning_rate="constant", solver="adam")
model_mlpclassifier_scaleddata.fit(X_scaled_train, y_train)

training_score = model_mlpclassifier_scaleddata.score(X_scaled_train, y_train)
print("Ergebnisse MLPClassifier auf skalierten Daten mit optimierten Parametern")
print(f"MLPClassifier Train score: {training_score:.3f}")

test_score = model_mlpclassifier_scaleddata.score(X_scaled_test, y_test)
print(f"MLPClassifier Test score: {test_score:.3f}")
print(f"MLPClassifier Train-Test score: {training_score-test_score}")

cross_val_scores = cross_val_score(model_mlpclassifier_scaleddata, X_scaled_train, y_train, cv=CV)
print(f"Cross-Value Scores: Mittelwert = {cross_val_scores.mean():.3f}, Std = {cross_val_scores.std():.3f}")
# Alt mit relu:
# MLPClassifier Train score: 1.000
# MLPClassifier Test score: 0.813
# Cross-Value Scores: Mittelwert = 0.739, Std = 0.027

Ergebnisse MLPClassifier auf skalierten Daten mit optimierten Parametern
MLPClassifier Train score: 0.999
MLPClassifier Test score: 0.826
MLPClassifier Train-Test score: 0.17367479745918812
Cross-Value Scores: Mittelwert = 0.757, Std = 0.009


# MLPClassifier mit GridSearch auf unskalierten Daten (nur zum Test)

In [20]:
# X_raw_train, X_raw_test, y_train, y_test = train_test_split(X_raw, y, test_size=0.2, random_state=42, shuffle=True)

# X_raw_train

In [21]:
# parameters = {
#     "solver":("lbfgs", "sgd", "adam"),
#     "hidden_layer_sizes":((100,), (2,2), (4,4), (6,6), (8,8), (10,10), (2,2,2), (4,4,4), (6,6,6), (12,12,12), (24,24,24)),  #, (2,2), (4,4), (6,6), (8,8), (10,10), (2,2,2), (4,4,4), (6,6,6), (12,12,12), (24,24,24)
#     "activation":("identity", "logistic", "tanh", "relu")
# }
# model_mlpclassifier = MLPClassifier(max_iter=10000, random_state=RANDOM_STATE)
# clf = GridSearchCV(model_mlpclassifier, parameters, n_jobs=N_JOBS, cv=CV)
# clf.fit(X_raw_train, y_train)
# clf_results = pd.DataFrame(clf.cv_results_)
# clf_results.sort_values(by="mean_test_score", ascending=False, inplace=True)
# clf_results

# MLPClassifier mit GridSearch auf PCA-behandelten Daten

In [26]:
with open('data/data_pca.pickle', 'rb') as f:
    df_pca = pickle.load(f)
X_pca = df_pca

C:\Users\sfremerey\AppData\Local\Temp\ipykernel_12088\560042475.py:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df_pca = pickle.load(f)


In [27]:
X_pca

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
-0.744657,-0.744657,-0.434217,0.613475,-0.038753,-0.651995,-0.486366,-0.525287,-0.040741,-0.017870,0.137689,-0.052642,-0.148739,-0.325130,-0.295340,0.283481,0.219684,-0.003914,0.201680,-0.076009,-0.228380
-0.944550,-0.944550,-0.467853,0.714482,-0.078727,-0.646375,-0.556171,-0.463230,0.070057,-0.046306,0.163363,-0.150614,-0.119672,-0.334759,-0.214427,0.310766,0.231164,-0.060912,0.198886,-0.061876,-0.231251
-1.121656,-1.121656,-0.542153,0.787781,-0.060076,-0.631773,-0.593225,-0.500995,0.104165,-0.048101,0.219705,-0.057761,-0.228854,-0.378949,-0.288156,0.275234,0.270031,-0.031478,0.104550,-0.109108,-0.200344
-1.219426,-1.219426,-0.485667,0.865095,-0.038853,-0.680765,-0.651743,-0.549390,0.088430,0.062574,0.270039,-0.071898,-0.274892,-0.265294,-0.384811,0.238427,0.213258,-0.066232,-0.077413,-0.239137,-0.217523
-1.284488,-1.284488,-0.167719,0.941966,-0.165316,-0.593576,-0.666616,-0.526511,0.118504,0.100745,0.355475,0.038305,-0.242886,-0.417864,-0.177324,0.338804,0.209904,0.140368,0.213606,-0.229046,-0.142161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-0.526201,-0.526201,0.192073,1.007247,-0.403304,0.911630,0.017375,-0.255465,-0.360773,0.398801,0.177015,-0.002211,-0.204771,0.396959,-0.069893,-0.034025,-0.271114,-0.299436,-0.307508,0.279217,-0.210137
-0.196996,-0.196996,0.215919,0.755334,-0.289580,0.987542,0.136987,-0.121914,-0.444564,0.395879,0.209484,-0.061677,-0.225493,0.159606,0.118206,-0.007055,-0.251972,-0.152396,0.031091,0.276431,-0.108809
-0.210521,-0.210521,0.470311,0.900156,-0.557417,0.864472,0.248264,-0.058227,-0.398830,0.600618,0.267994,0.154195,-0.335089,0.281525,0.131572,-0.047605,-0.137308,-0.163758,0.029635,0.544831,-0.157099
-0.070200,-0.070200,0.489911,0.818974,-0.520897,0.868750,0.330206,-0.035543,-0.560524,0.390786,0.265077,0.154407,-0.300362,0.111722,0.269529,0.059834,-0.159962,-0.110877,0.180398,0.302989,-0.247769


In [28]:
X_pca_train, X_pca_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42, shuffle=True)

In [29]:
parameters = {
    "solver":("lbfgs", "sgd", "adam"),
    "hidden_layer_sizes":((100,), (2,2), (4,4), (6,6), (8,8), (10,10), (2,2,2), (4,4,4), (6,6,6), (12,12,12), (24,24,24)),  #, (2,2), (4,4), (6,6), (8,8), (10,10), (2,2,2), (4,4,4), (6,6,6), (12,12,12), (24,24,24)
    "activation":("identity", "logistic", "tanh", "relu"),
    "learning_rate":(("constant", "invscaling", "adaptive"))
}
model_mlpclassifier = MLPClassifier(max_iter=10000, random_state=RANDOM_STATE)
clf = GridSearchCV(model_mlpclassifier, parameters, n_jobs=N_JOBS, cv=CV)
clf.fit(X_pca_train, y_train)
clf_results = pd.DataFrame(clf.cv_results_)
clf_results.sort_values(by="mean_test_score", ascending=False, inplace=True)
# mean_test_score für source data X_pca_scaled: 0.701234 (schlechter, deshalb nicht genommen)
# mean_test_score für source data X_pca: 0.758202
clf_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_hidden_layer_sizes,param_learning_rate,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
203,56.330474,1.974933,0.013499,0.002728,tanh,"(100,)",invscaling,adam,"{'activation': 'tanh', 'hidden_layer_sizes': (...",0.735409,0.766537,0.757812,0.777344,0.753906,0.758202,0.013957,1
206,62.935271,1.913002,0.012158,0.004516,tanh,"(100,)",adaptive,adam,"{'activation': 'tanh', 'hidden_layer_sizes': (...",0.735409,0.766537,0.757812,0.777344,0.753906,0.758202,0.013957,1
200,57.317401,2.380958,0.011287,0.001293,tanh,"(100,)",constant,adam,"{'activation': 'tanh', 'hidden_layer_sizes': (...",0.735409,0.766537,0.757812,0.777344,0.753906,0.758202,0.013957,1
305,29.141881,2.409600,0.009962,0.002412,relu,"(100,)",adaptive,adam,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.750973,0.747082,0.742188,0.777344,0.707031,0.744923,0.022519,4
302,28.598241,2.826189,0.012859,0.003334,relu,"(100,)",invscaling,adam,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.750973,0.747082,0.742188,0.777344,0.707031,0.744923,0.022519,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,0.460324,0.089895,0.008023,0.001073,logistic,"(10, 10)",invscaling,sgd,"{'activation': 'logistic', 'hidden_layer_sizes...",0.291829,0.291829,0.289062,0.289062,0.289062,0.290169,0.001355,389
256,0.232081,0.023141,0.007213,0.001051,tanh,"(2, 2, 2)",invscaling,sgd,"{'activation': 'tanh', 'hidden_layer_sizes': (...",0.291829,0.291829,0.289062,0.289062,0.289062,0.290169,0.001355,389
391,0.380938,0.060259,0.007395,0.001537,relu,"(24, 24, 24)",invscaling,sgd,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.299611,0.303502,0.285156,0.277344,0.281250,0.289373,0.010324,394
22,1.479882,0.235832,0.009457,0.005050,identity,"(4, 4)",invscaling,sgd,"{'activation': 'identity', 'hidden_layer_sizes...",0.307393,0.280156,0.277344,0.285156,0.281250,0.286260,0.010860,395


In [30]:
model_mlpclassifier_pcadata = MLPClassifier(max_iter=10000, random_state=RANDOM_STATE, hidden_layer_sizes=(100,), activation="tanh", solver="adam")
model_mlpclassifier_pcadata.fit(X_pca_train, y_train)

training_score = model_mlpclassifier_pcadata.score(X_pca_train, y_train)
print("Ergebnisse MLPClassifier auf PCA-behandelten Daten mit optimierten Parametern")
print(f"MLPClassifier Train score: {training_score:.3f}")

test_score = model_mlpclassifier_pcadata.score(X_pca_test, y_test)
print(f"MLPClassifier Test score: {test_score:.3f}")
print(f"MLPClassifier Train-Test score: {training_score-test_score}")

cross_val_scores = cross_val_score(model_mlpclassifier_pcadata, X_pca_train, y_train, cv=CV)
print(f"Cross-Value Scores: Mittelwert = {cross_val_scores.mean():.3f}, Std = {cross_val_scores.std():.3f}")

Ergebnisse MLPClassifier auf PCA-behandelten Daten mit optimierten Parametern
MLPClassifier Train score: 1.000
MLPClassifier Test score: 0.804
MLPClassifier Train-Test score: 0.19626168224299068
Cross-Value Scores: Mittelwert = 0.758, Std = 0.014


# KNeighborsClassifier mit GridSearch auf skalierten Daten

In [31]:
parameters = {
    "n_neighbors": (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20),
    "weights":("uniform", "distance"),
    "algorithm":("auto", "ball_tree", "kd_tree", "brute"),
    "leaf_size":(15, 20, 30, 40, 50, 60, 90),
    "p":(1, 2, 3, 4, 5, 6, 7, 8, 9, 10),
}
model_kneighbors = KNeighborsClassifier(n_jobs=N_JOBS)
clf = GridSearchCV(model_kneighbors, parameters, n_jobs=N_JOBS, cv=CV)
clf.fit(X_scaled_train, y_train)
clf_results = pd.DataFrame(clf.cv_results_)
clf_results.sort_values(by="mean_test_score", ascending=False, inplace=True)
# mean_test_score: 0.794084
clf_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_leaf_size,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
6,0.050923,0.031438,1.208054,0.299353,auto,15,1,4,uniform,"{'algorithm': 'auto', 'leaf_size': 15, 'n_neig...",0.805447,0.766537,0.796875,0.832031,0.769531,0.794084,0.024243,1
7,0.015303,0.004828,1.232724,0.410104,auto,15,1,4,distance,"{'algorithm': 'auto', 'leaf_size': 15, 'n_neig...",0.805447,0.766537,0.796875,0.832031,0.769531,0.794084,0.024243,1
827,0.009607,0.004035,0.547619,0.022561,auto,30,2,4,distance,"{'algorithm': 'auto', 'leaf_size': 30, 'n_neig...",0.805447,0.766537,0.796875,0.832031,0.769531,0.794084,0.024243,1
1206,0.012033,0.002571,0.579377,0.036519,auto,40,1,4,uniform,"{'algorithm': 'auto', 'leaf_size': 40, 'n_neig...",0.805447,0.766537,0.796875,0.832031,0.769531,0.794084,0.024243,1
1207,0.007979,0.000449,0.502839,0.016525,auto,40,1,4,distance,"{'algorithm': 'auto', 'leaf_size': 40, 'n_neig...",0.805447,0.766537,0.796875,0.832031,0.769531,0.794084,0.024243,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7998,0.066314,0.022751,0.427529,0.057039,kd_tree,60,20,10,uniform,"{'algorithm': 'kd_tree', 'leaf_size': 60, 'n_n...",0.552529,0.552529,0.554688,0.582031,0.542969,0.556949,0.013181,11173
9998,0.008518,0.001475,0.538401,0.029722,brute,40,20,10,uniform,"{'algorithm': 'brute', 'leaf_size': 40, 'n_nei...",0.552529,0.552529,0.554688,0.582031,0.542969,0.556949,0.013181,11173
6798,0.063767,0.007039,0.428718,0.023437,kd_tree,30,20,10,uniform,"{'algorithm': 'kd_tree', 'leaf_size': 30, 'n_n...",0.552529,0.552529,0.554688,0.582031,0.542969,0.556949,0.013181,11173
398,0.031066,0.020463,1.135937,0.073072,auto,15,20,10,uniform,"{'algorithm': 'auto', 'leaf_size': 15, 'n_neig...",0.552529,0.552529,0.554688,0.582031,0.542969,0.556949,0.013181,11173


In [32]:
model_kneighbors_scaleddata = KNeighborsClassifier(n_jobs=N_JOBS, algorithm="auto", leaf_size=15, p=4, weights="uniform", n_neighbors=1)
model_kneighbors_scaleddata.fit(X_scaled_train, y_train)
print("Ergebnisse KNeighborsClassifier auf skalierten Daten mit optimierten Parametern")
training_score = model_kneighbors_scaleddata.score(X_scaled_train, y_train)
print(f"KNeighborsClassifier Train scores: {training_score}")
test_score = model_kneighbors_scaleddata.score(X_scaled_test, y_test)
print(f"KNeighborsClassifier Test scores: {test_score}")
print(f"KNeighborsClassifier Train-Test score: {training_score-test_score}")
model_kneighbors_scores = cross_val_score(model_kneighbors_scaleddata, X_scaled_train, y_train, cv=CV)
print(f"Cross-Value Scores: Mittelwert = {model_kneighbors_scores.mean():.3f}, Std = {model_kneighbors_scores.std():.3f}")

Ergebnisse KNeighborsClassifier auf skalierten Daten mit optimierten Parametern
KNeighborsClassifier Train scores: 1.0
KNeighborsClassifier Test scores: 0.7975077881619937
KNeighborsClassifier Train-Test score: 0.20249221183800625
Cross-Value Scores: Mittelwert = 0.794, Std = 0.024


# KNeighborsClassifier mit und ohne GridSearch auf PCA-behandelten Daten

In [33]:
parameters = {
    "n_neighbors": (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20),
    "weights":("uniform", "distance"),
    "algorithm":("auto", "ball_tree", "kd_tree", "brute"),
    "leaf_size":(15, 20, 30, 40, 50, 60, 90),
    "p":(1, 2, 3, 4, 5, 6, 7, 8, 9, 10),
}
model_kneighbors = KNeighborsClassifier(n_jobs=N_JOBS)
clf = GridSearchCV(model_kneighbors, parameters, n_jobs=N_JOBS, cv=CV)
clf.fit(X_pca_train, y_train)
clf_results = pd.DataFrame(clf.cv_results_)
clf_results.sort_values(by="mean_test_score", ascending=False, inplace=True)
# mean_test_score: 0.792525
clf_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_leaf_size,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10823,0.004423,0.000651,0.009260,0.000350,brute,90,2,2,distance,"{'algorithm': 'brute', 'leaf_size': 90, 'n_nei...",0.801556,0.770428,0.800781,0.816406,0.78125,0.794084,0.016268,1
23,0.005382,0.002232,0.008789,0.002022,auto,15,2,2,distance,"{'algorithm': 'auto', 'leaf_size': 15, 'n_neig...",0.801556,0.770428,0.800781,0.816406,0.78125,0.794084,0.016268,1
10803,0.004002,0.001136,0.008046,0.001441,brute,90,1,2,distance,"{'algorithm': 'brute', 'leaf_size': 90, 'n_nei...",0.801556,0.770428,0.800781,0.816406,0.78125,0.794084,0.016268,1
10802,0.003741,0.000612,0.010147,0.001214,brute,90,1,2,uniform,"{'algorithm': 'brute', 'leaf_size': 90, 'n_nei...",0.801556,0.770428,0.800781,0.816406,0.78125,0.794084,0.016268,1
10423,0.006975,0.000443,0.012168,0.000732,brute,60,2,2,distance,"{'algorithm': 'brute', 'leaf_size': 60, 'n_nei...",0.801556,0.770428,0.800781,0.816406,0.78125,0.794084,0.016268,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1986,0.004223,0.000851,0.265851,0.030482,auto,50,20,4,uniform,"{'algorithm': 'auto', 'leaf_size': 50, 'n_neig...",0.583658,0.544747,0.582031,0.593750,0.59375,0.579587,0.018097,11173
3186,0.025794,0.004261,0.246007,0.019292,ball_tree,15,20,4,uniform,"{'algorithm': 'ball_tree', 'leaf_size': 15, 'n...",0.583658,0.544747,0.582031,0.593750,0.59375,0.579587,0.018097,11173
6386,0.022390,0.007964,0.155757,0.011490,kd_tree,20,20,4,uniform,"{'algorithm': 'kd_tree', 'leaf_size': 20, 'n_n...",0.583658,0.544747,0.582031,0.593750,0.59375,0.579587,0.018097,11173
5586,0.012897,0.002704,0.266564,0.019239,ball_tree,90,20,4,uniform,"{'algorithm': 'ball_tree', 'leaf_size': 90, 'n...",0.583658,0.544747,0.582031,0.593750,0.59375,0.579587,0.018097,11173


In [34]:
model_kneighbors_pcadata = KNeighborsClassifier(n_jobs=N_JOBS, algorithm="auto", leaf_size=15, p=2, weights="distance", n_neighbors=2)
model_kneighbors_pcadata.fit(X_pca_train, y_train)
print("Ergebnisse KNeighborsClassifier auf PCA-behandelten Daten mit optimierten Parametern")
training_score = model_kneighbors_pcadata.score(X_pca_train, y_train)
print(f"KNeighborsClassifier Train scores: {training_score}")
test_score = model_kneighbors_pcadata.score(X_pca_test, y_test)
print(f"KNeighborsClassifier Test scores: {test_score}")
print(f"KNeighborsClassifier Train-Test score: {training_score-test_score}")
model_kneighbors_scores = cross_val_score(model_kneighbors_pcadata, X_pca_train, y_train, cv=CV)
print(f"Cross-Value Scores: Mittelwert = {model_kneighbors_scores.mean():.3f}, Std = {model_kneighbors_scores.std():.3f}")

Ergebnisse KNeighborsClassifier auf PCA-behandelten Daten mit optimierten Parametern
KNeighborsClassifier Train scores: 1.0
KNeighborsClassifier Test scores: 0.8130841121495327
KNeighborsClassifier Train-Test score: 0.1869158878504673
Cross-Value Scores: Mittelwert = 0.794, Std = 0.016


# RadiusNeighborsClassifier mit GridSearch auf skalierten Daten

In [35]:
parameters = {
    "radius": (1.35, 1.351, 1.352, 1.353, 1.354, 1.355, 1.36, 1.38, 1.4, 1.5, 1.75, 2, 3, 5, 10, 15, 20, 30, 50),
    "weights":("uniform", "distance"),
    "algorithm":("auto", "ball_tree", "kd_tree", "brute"),
    "leaf_size":(15, 20, 30, 40, 50, 60, 90),
    "p":(1, 2, 3, 4, 5, 6, 7, 8, 9, 10),
}
model_kneighborsradius = RadiusNeighborsClassifier(n_jobs=N_JOBS, radius=1.35)
clf = GridSearchCV(model_kneighborsradius, parameters, n_jobs=N_JOBS, cv=CV)
clf.fit(X_scaled_train, y_train)
clf_results = pd.DataFrame(clf.cv_results_)
clf_results.sort_values(by="mean_test_score", ascending=False, inplace=True)
clf_results

C:\Users\sfremerey\.conda\envs\py3135\Lib\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan ... 0.38221972 0.36895367 0.38221972]
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_leaf_size,param_p,param_radius,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
6881,0.035507,0.010324,0.201913,0.022581,kd_tree,50,2,1.351,distance,"{'algorithm': 'kd_tree', 'leaf_size': 50, 'p':...",0.626459,0.603113,0.597656,0.640625,0.593750,0.612321,0.018142,1
6879,0.032519,0.006580,0.172808,0.028106,kd_tree,50,2,1.350,distance,"{'algorithm': 'kd_tree', 'leaf_size': 50, 'p':...",0.626459,0.603113,0.593750,0.644531,0.593750,0.612321,0.020060,1
10303,0.011078,0.000566,0.020796,0.001342,brute,90,2,1.352,distance,"{'algorithm': 'brute', 'leaf_size': 90, 'p': 2...",0.630350,0.599222,0.597656,0.640625,0.593750,0.612321,0.019275,1
6883,0.032758,0.008556,0.199704,0.023275,kd_tree,50,2,1.352,distance,"{'algorithm': 'kd_tree', 'leaf_size': 50, 'p':...",0.630350,0.599222,0.597656,0.640625,0.593750,0.612321,0.019275,1
6499,0.031655,0.007418,0.250175,0.027316,kd_tree,40,2,1.350,distance,"{'algorithm': 'kd_tree', 'leaf_size': 40, 'p':...",0.626459,0.603113,0.593750,0.644531,0.593750,0.612321,0.020060,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10283,0.009522,0.000554,0.026689,0.001441,brute,90,1,2.000,distance,"{'algorithm': 'brute', 'leaf_size': 90, 'p': 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9857
10284,0.008702,0.000646,0.025582,0.001902,brute,90,1,3.000,uniform,"{'algorithm': 'brute', 'leaf_size': 90, 'p': 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9857
10285,0.008795,0.000949,0.026825,0.001360,brute,90,1,3.000,distance,"{'algorithm': 'brute', 'leaf_size': 90, 'p': 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9857
10286,0.009163,0.001461,0.029943,0.001501,brute,90,1,5.000,uniform,"{'algorithm': 'brute', 'leaf_size': 90, 'p': 1...",0.607004,0.665370,NaN,0.632812,0.632812,NaN,NaN,9857


In [36]:
model_kneighborsradius_scaleddata = RadiusNeighborsClassifier(n_jobs=N_JOBS, algorithm="kd_tree", leaf_size=50, p=2, radius=1.351, weights="distance")
model_kneighborsradius_scaleddata.fit(X_scaled_train, y_train)
print("Ergebnisse RadiusNeighborsClassifier auf skalierten Daten mit optimierten Parametern")
training_score = model_kneighborsradius_scaleddata.score(X_scaled_train, y_train)
print(f"RadiusNeighborsClassifier Train scores: {training_score}")
test_score = model_kneighborsradius_scaleddata.score(X_scaled_test, y_test)
print(f"RadiusNeighborsClassifier Test scores: {test_score}")
print(f"RadiusNeighborsClassifier Train-Test score: {training_score-test_score}")
model_kneighborsradius_scores = cross_val_score(model_kneighborsradius_scaleddata, X_scaled_train, y_train, cv=CV)
print(f"Cross-Value Scores: Mittelwert = {model_kneighborsradius_scores.mean():.3f}, Std = {model_kneighborsradius_scores.std():.3f}")

Ergebnisse RadiusNeighborsClassifier auf skalierten Daten mit optimierten Parametern
RadiusNeighborsClassifier Train scores: 1.0
RadiusNeighborsClassifier Test scores: 0.67601246105919
RadiusNeighborsClassifier Train-Test score: 0.32398753894081
Cross-Value Scores: Mittelwert = 0.612, Std = 0.018


# RadiusNeighborsClassifier mit GridSearch auf PCA-behandelten Daten

In [37]:
parameters = {
    "radius": (1.35, 1.351, 1.352, 1.353, 1.354, 1.355, 1.36, 1.38, 1.4, 1.5, 1.75, 2, 3, 5, 10, 15, 20, 30, 50),
    "weights":("uniform", "distance"),
    "algorithm":("auto", "ball_tree", "kd_tree", "brute"),
    "leaf_size":(15, 20, 30, 40, 50, 60, 90),
    "p":(1, 2, 3, 4, 5, 6, 7, 8, 9, 10),
}
model_kneighborsradius = RadiusNeighborsClassifier(n_jobs=N_JOBS, radius=1.35)
clf = GridSearchCV(model_kneighborsradius, parameters, n_jobs=N_JOBS, cv=CV)
clf.fit(X_pca_train, y_train)
clf_results = pd.DataFrame(clf.cv_results_)
clf_results.sort_values(by="mean_test_score", ascending=False, inplace=True)
clf_results

C:\Users\sfremerey\.conda\envs\py3135\Lib\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan ... 0.43761248 0.36895367 0.43761248]
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_leaf_size,param_p,param_radius,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
55,0.006484,0.000870,0.024000,0.000956,auto,15,2,1.40,distance,"{'algorithm': 'auto', 'leaf_size': 15, 'p': 2,...",0.622568,0.579767,0.5625,0.621094,0.554688,0.588123,0.028698,1
2335,0.020204,0.015899,0.055704,0.019091,auto,90,2,1.40,distance,"{'algorithm': 'auto', 'leaf_size': 90, 'p': 2,...",0.622568,0.579767,0.5625,0.621094,0.554688,0.588123,0.028698,1
8035,0.008760,0.005025,0.026879,0.009508,brute,15,2,1.40,distance,"{'algorithm': 'brute', 'leaf_size': 15, 'p': 2...",0.622568,0.579767,0.5625,0.621094,0.554688,0.588123,0.028698,1
8415,0.009522,0.002997,0.047233,0.016803,brute,20,2,1.40,distance,"{'algorithm': 'brute', 'leaf_size': 20, 'p': 2...",0.622568,0.579767,0.5625,0.621094,0.554688,0.588123,0.028698,1
1575,0.008429,0.002317,0.027631,0.001925,auto,50,2,1.40,distance,"{'algorithm': 'auto', 'leaf_size': 50, 'p': 2,...",0.622568,0.579767,0.5625,0.621094,0.554688,0.588123,0.028698,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10281,0.004927,0.000417,0.024094,0.002797,brute,90,1,1.75,distance,"{'algorithm': 'brute', 'leaf_size': 90, 'p': 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9913
10282,0.006734,0.001075,0.021550,0.002294,brute,90,1,2.00,uniform,"{'algorithm': 'brute', 'leaf_size': 90, 'p': 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9913
10283,0.005781,0.000957,0.021329,0.001687,brute,90,1,2.00,distance,"{'algorithm': 'brute', 'leaf_size': 90, 'p': 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9913
10284,0.005538,0.000348,0.024321,0.001755,brute,90,1,3.00,uniform,"{'algorithm': 'brute', 'leaf_size': 90, 'p': 1...",0.657588,NaN,NaN,NaN,0.699219,NaN,NaN,9913


In [38]:
model_kneighborsradius_pcadata = RadiusNeighborsClassifier(n_jobs=N_JOBS, algorithm="auto", leaf_size=15, p=2, radius=1.4, weights="distance")
model_kneighborsradius_pcadata.fit(X_pca_train, y_train)
print("Ergebnisse RadiusNeighborsClassifier auf PCA-behandelten Daten mit optimierten Parametern")
training_score = model_kneighborsradius_pcadata.score(X_pca_train, y_train)
print(f"RadiusNeighborsClassifier Train scores: {training_score}")
test_score = model_kneighborsradius_pcadata.score(X_pca_test, y_test)
print(f"RadiusNeighborsClassifier Test scores: {test_score}")
print(f"RadiusNeighborsClassifier Train-Test score: {training_score-test_score}")
model_kneighborsradius_scores = cross_val_score(model_kneighborsradius_pcadata, X_pca_train, y_train, cv=CV)
print(f"Cross-Value Scores: Mittelwert = {model_kneighborsradius_scores.mean():.3f}, Std = {model_kneighborsradius_scores.std():.3f}")

Ergebnisse RadiusNeighborsClassifier auf PCA-behandelten Daten mit optimierten Parametern
RadiusNeighborsClassifier Train scores: 1.0
RadiusNeighborsClassifier Test scores: 0.6573208722741433
RadiusNeighborsClassifier Train-Test score: 0.34267912772585674
Cross-Value Scores: Mittelwert = 0.588, Std = 0.029


# GaussianNB auf skalierten Daten

In [39]:
model_gaussiannb_scaleddata = GaussianNB()
model_gaussiannb_scaleddata.fit(X_scaled_train, y_train)
print("Ergebnisse GaussianNB auf skalierten Daten")
training_score = model_gaussiannb_scaleddata.score(X_scaled_train, y_train)
print(f"GaussianNB Train scores: {training_score}")
test_score = model_gaussiannb_scaleddata.score(X_scaled_test, y_test)
print(f"GaussianNB Test scores: {test_score}")
print(f"GaussianNB Train-Test score: {training_score-test_score}")
model_gaussiannb_scores = cross_val_score(model_gaussiannb_scaleddata, X_scaled_train, y_train, cv=CV)
print(f"Cross-Value Scores: Mittelwert = {model_gaussiannb_scores.mean():.3f}, Std = {model_gaussiannb_scores.std():.3f}")

Ergebnisse GaussianNB auf skalierten Daten
GaussianNB Train scores: 0.5015600624024961
GaussianNB Test scores: 0.5077881619937694
GaussianNB Train-Test score: -0.0062280995912733506
Cross-Value Scores: Mittelwert = 0.455, Std = 0.037


# GaussianNB auf PCA-behandelten Daten

In [40]:
model_gaussiannb_pcadata = GaussianNB()
model_gaussiannb_pcadata.fit(X_pca_train, y_train)
print("Ergebnisse GaussianNB auf PCA-behandelten Daten")
training_score = model_gaussiannb_pcadata.score(X_pca_train, y_train)
print(f"GaussianNB Train scores: {training_score}")
test_score = model_gaussiannb_pcadata.score(X_pca_test, y_test)
print(f"GaussianNB Test scores: {test_score}")
print(f"GaussianNB Train-Test score: {training_score-test_score}")
model_gaussiannb_scores = cross_val_score(model_gaussiannb_pcadata, X_pca_train, y_train, cv=CV)
print(f"Cross-Value Scores: Mittelwert = {model_gaussiannb_scores.mean():.3f}, Std = {model_gaussiannb_scores.std():.3f}")

Ergebnisse GaussianNB auf PCA-behandelten Daten
GaussianNB Train scores: 0.5023400936037441
GaussianNB Test scores: 0.48598130841121495
GaussianNB Train-Test score: 0.016358785192529146
Cross-Value Scores: Mittelwert = 0.466, Std = 0.025
